# Candlestick Pattern Recognition

This notebook demonstrates how to use numta's candlestick pattern recognition functions.

In [ ]:
import numpy as np
import pandas as pd
import numta
from numta import (
    # Single candle patterns
    CDLDOJI, CDLHAMMER, CDLSHOOTINGSTAR, CDLMARUBOZU, CDLSPINNINGTOP,
    CDLINVERTEDHAMMER, CDLHANGINGMAN, CDLDRAGONFLYDOJI, CDLGRAVESTONEDOJI,
    # Double candle patterns
    CDLENGULFING, CDLHARAMI, CDLHARAMICROSS, CDLPIERCING, CDLDARKCLOUDCOVER,
    # Triple candle patterns
    CDLMORNINGSTAR, CDLEVENINGSTAR, CDL3WHITESOLDIERS, CDL3BLACKCROWS,
    CDLMORNINGDOJISTAR, CDLEVENINGDOJISTAR
)

## Creating Sample Data

We'll create OHLC data with some embedded patterns for demonstration.

In [ ]:
# Create sample OHLCV data
np.random.seed(42)
n = 100

# Generate random walk prices
base_price = 100
returns = np.random.randn(n) * 0.02
close = base_price * np.cumprod(1 + returns)

# Generate OHLC from close
open_ = np.roll(close, 1)
open_[0] = close[0]

# Add some randomness to high/low
high = np.maximum(open_, close) * (1 + np.abs(np.random.randn(n) * 0.01))
low = np.minimum(open_, close) * (1 - np.abs(np.random.randn(n) * 0.01))

df = pd.DataFrame({
    'open': open_,
    'high': high,
    'low': low,
    'close': close
})

print(df.head(10))

## Introduction to Candlestick Patterns

Candlestick patterns are formations of one or more candles that signal potential price reversals or continuations.

Pattern functions return:
- **+100**: Bullish pattern detected
- **-100**: Bearish pattern detected
- **0**: No pattern detected

## Single Candle Patterns

### Doji

A doji occurs when the open and close prices are virtually equal. It indicates indecision in the market.

In [ ]:
doji = CDLDOJI(open_, high, low, close)
doji_signals = np.where(doji != 0)[0]
print(f"Doji patterns found at indices: {doji_signals[:10]}")

### Hammer and Shooting Star

- **Hammer**: Bullish reversal pattern with small body and long lower shadow
- **Shooting Star**: Bearish reversal pattern with small body and long upper shadow

In [ ]:
hammer = CDLHAMMER(open_, high, low, close)
shooting_star = CDLSHOOTINGSTAR(open_, high, low, close)

print(f"Hammer signals: {np.sum(hammer == 100)} bullish")
print(f"Shooting Star signals: {np.sum(shooting_star == -100)} bearish")

### Marubozu

A marubozu is a candle with no shadows (wicks), indicating strong conviction.

In [ ]:
marubozu = CDLMARUBOZU(open_, high, low, close)
bullish = np.sum(marubozu == 100)
bearish = np.sum(marubozu == -100)
print(f"Marubozu: {bullish} bullish, {bearish} bearish")

### Doji Variants

In [ ]:
# Dragonfly Doji - long lower shadow, no upper shadow (bullish)
dragonfly = CDLDRAGONFLYDOJI(open_, high, low, close)

# Gravestone Doji - long upper shadow, no lower shadow (bearish)
gravestone = CDLGRAVESTONEDOJI(open_, high, low, close)

print(f"Dragonfly Doji (bullish): {np.sum(dragonfly != 0)}")
print(f"Gravestone Doji (bearish): {np.sum(gravestone != 0)}")

## Double Candle Patterns

### Engulfing Pattern

The second candle completely engulfs the body of the first candle.

In [ ]:
engulfing = CDLENGULFING(open_, high, low, close)
bullish = np.sum(engulfing == 100)
bearish = np.sum(engulfing == -100)
print(f"Engulfing patterns: {bullish} bullish, {bearish} bearish")

### Harami Pattern

The second candle's body is contained within the first candle's body.

In [ ]:
harami = CDLHARAMI(open_, high, low, close)
harami_cross = CDLHARAMICROSS(open_, high, low, close)

print(f"Harami: {np.sum(harami == 100)} bullish, {np.sum(harami == -100)} bearish")
print(f"Harami Cross: {np.sum(harami_cross != 0)} signals")

### Piercing and Dark Cloud Cover

In [ ]:
# Piercing - bullish reversal
piercing = CDLPIERCING(open_, high, low, close)

# Dark Cloud Cover - bearish reversal
dark_cloud = CDLDARKCLOUDCOVER(open_, high, low, close)

print(f"Piercing (bullish): {np.sum(piercing == 100)}")
print(f"Dark Cloud Cover (bearish): {np.sum(dark_cloud == -100)}")

## Triple Candle Patterns

### Morning Star and Evening Star

- **Morning Star**: Bullish reversal - large bearish, small body, large bullish
- **Evening Star**: Bearish reversal - large bullish, small body, large bearish

In [ ]:
morning_star = CDLMORNINGSTAR(open_, high, low, close, penetration=0.3)
evening_star = CDLEVENINGSTAR(open_, high, low, close, penetration=0.3)

print(f"Morning Star (bullish): {np.sum(morning_star == 100)}")
print(f"Evening Star (bearish): {np.sum(evening_star == -100)}")

### Three White Soldiers and Three Black Crows

In [ ]:
# Three White Soldiers - strong bullish
soldiers = CDL3WHITESOLDIERS(open_, high, low, close)

# Three Black Crows - strong bearish
crows = CDL3BLACKCROWS(open_, high, low, close)

print(f"Three White Soldiers (bullish): {np.sum(soldiers == 100)}")
print(f"Three Black Crows (bearish): {np.sum(crows == -100)}")

## Using Patterns for Trading Signals

Here's how you might combine multiple patterns to generate trading signals.

In [ ]:
# Calculate multiple patterns
patterns_df = pd.DataFrame({
    'close': close,
    'doji': CDLDOJI(open_, high, low, close),
    'hammer': CDLHAMMER(open_, high, low, close),
    'engulfing': CDLENGULFING(open_, high, low, close),
    'morning_star': CDLMORNINGSTAR(open_, high, low, close),
    'evening_star': CDLEVENINGSTAR(open_, high, low, close)
})

# Create aggregate signals
patterns_df['bullish_signal'] = (patterns_df[['hammer', 'engulfing', 'morning_star']] > 0).any(axis=1)
patterns_df['bearish_signal'] = (patterns_df[['engulfing', 'evening_star']] < 0).any(axis=1)

print("Pattern summary:")
print(f"  Total bullish signals: {patterns_df['bullish_signal'].sum()}")
print(f"  Total bearish signals: {patterns_df['bearish_signal'].sum()}")

## Using the Pandas Accessor

Pattern functions are also available through the `.ta` accessor.

In [ ]:
# Reset dataframe
df = pd.DataFrame({
    'open': open_,
    'high': high,
    'low': low,
    'close': close
})

# Add pattern columns
df.ta.cdldoji(append=True)
df.ta.cdlengulfing(append=True)
df.ta.cdlhammer(append=True)

print(df.columns.tolist())

## All Available Candlestick Patterns

numta supports 60+ candlestick patterns. Here are the main categories:

### Single Candle
- CDLDOJI, CDLHAMMER, CDLSHOOTINGSTAR, CDLMARUBOZU, CDLSPINNINGTOP
- CDLINVERTEDHAMMER, CDLHANGINGMAN, CDLDRAGONFLYDOJI, CDLGRAVESTONEDOJI
- CDLLONGLINE, CDLSHORTLINE, CDLRICKSHAWMAN, CDLTAKURI, CDLHIGHWAVE

### Double Candle
- CDLENGULFING, CDLHARAMI, CDLHARAMICROSS, CDLPIERCING, CDLDARKCLOUDCOVER
- CDLCOUNTERATTACK, CDLKICKING, CDLMATCHINGLOW, CDLHOMINGPIGEON

### Triple Candle
- CDLMORNINGSTAR, CDLEVENINGSTAR, CDL3WHITESOLDIERS, CDL3BLACKCROWS
- CDLMORNINGDOJISTAR, CDLEVENINGDOJISTAR, CDL3INSIDE, CDL3OUTSIDE
- CDLABANDONEDBABY, CDLTRISTAR

See the full list in [FUNCTION_IMPLEMENTATIONS.md](../FUNCTION_IMPLEMENTATIONS.md).

## Next Steps

- See `04_chart_patterns.ipynb` for chart pattern detection (head & shoulders, double tops, etc.)
- See `05_harmonic_patterns.ipynb` for harmonic pattern recognition
- See `07_visualization.ipynb` for visualizing patterns with lwcharts